# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
from spicy import stats
from statistics import mean

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrety import IEX_CLOUD_API_TOKEN
stocks

,Ticker
0,MMM
1,AOS
2,ABT
3,ABBV
4,ACN
...,...
498,XYL
499,YUM
500,ZBRA
501,ZBH


## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
 # /stock/{symbol}/stats/{stat?}
symbol = 'AAPL'
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data =  requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2909182037040,
 'week52high': 198.57,
 'week52low': 163.63,
 'week52highSplitAdjustOnly': 199.62,
 'week52lowSplitAdjustOnly': 164.08,
 'week52change': 0.11031453644719248,
 'sharesOutstanding': 15334082000,
 'float': 0,
 'avg10Volume': 59563392,
 'avg30Volume': 63853142,
 'day200MovingAvg': 181.5,
 'day50MovingAvg': 173.31,
 'employees': 164000,
 'ttmEPS': 6.43,
 'ttmDividendRate': 0.966204324874062,
 'dividendYield': 0.005092791086201043,
 'nextDividendDate': '',
 'exDividendDate': '2024-05-10',
 'nextEarningsDate': '2024-08-01',
 'peRatio': 28.979091703672715,
 'beta': 1.05132076794146,
 'maxChangePercent': 73.54616895874264,
 'year5ChangePercent': 3.145707230732251,
 'year2ChangePercent': 0.3128176171654433,
 'year1ChangePercent': 0.11431476279109654,
 'ytdChangePercent': -0.00939797900896988,
 'month6ChangePercent': 0.01441743181074484,
 'month3ChangePercent': 0.034672321042918286,
 'month1ChangePercent': 0.10159758359239723,
 'day30Chang

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data['year1ChangePercent']

0.11431476279109654

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings[:1]:
    # batch_api_call_url = ''
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    print(data['AAPL'])
    for symbol in symbol_string.split(','):
        new_data = {'Ticker': symbol, 
        'Price': data[symbol]['quote']['latestPrice'], 
        'One-Year Price Return': data[symbol]['stats']['year1ChangePercent'], 
        'Number of Shares to Buy': 'N/A'}

        # Get the index for the new row
        new_index = len(final_dataframe)
        
        # Set the values for each column at the new index
        for column, value in new_data.items():
            final_dataframe.at[new_index, column] = value

final_dataframe

{'stats': {'companyName': 'Apple Inc', 'marketcap': 2909182037040, 'week52high': 198.57, 'week52low': 163.63, 'week52highSplitAdjustOnly': 199.62, 'week52lowSplitAdjustOnly': 164.08, 'week52change': 0.11031453644719248, 'sharesOutstanding': 15334082000, 'float': 0, 'avg10Volume': 59563392, 'avg30Volume': 63853142, 'day200MovingAvg': 181.5, 'day50MovingAvg': 173.31, 'employees': 164000, 'ttmEPS': 6.43, 'ttmDividendRate': 0.966204324874062, 'dividendYield': 0.005092791086201043, 'nextDividendDate': '', 'exDividendDate': '2024-05-10', 'nextEarningsDate': '2024-08-01', 'peRatio': 28.979091703672715, 'beta': 1.05132076794146, 'maxChangePercent': 73.54616895874264, 'year5ChangePercent': 3.145707230732251, 'year2ChangePercent': 0.3128176171654433, 'year1ChangePercent': 0.11431476279109654, 'ytdChangePercent': -0.00939797900896988, 'month6ChangePercent': 0.01441743181074484, 'month3ChangePercent': 0.034672321042918286, 'month1ChangePercent': 0.10159758359239723, 'day30ChangePercent': 0.1015975

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,MMM,104.86,0.071923,N/A
1,AOS,85.36,0.293571,N/A
2,ABT,104.87,-0.028491,N/A
3,ABBV,164.35,0.162688,N/A
4,ACN,308,0.128852,N/A
...,...,...,...,...
95,CE,155.67,0.53876,N/A
96,COR,221.85,0.282975,N/A
97,CNC,77.14,0.161881,N/A
98,CNP,29.94,0.044046,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [7]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,44,ANET,320.09,1.362879,N/A
1,75,AVGO,1412.13,1.321727,N/A
2,40,AMAT,214.03,0.822188,N/A
3,34,APH,131.71,0.758475,N/A
4,91,CAT,350.72,0.735574,N/A
5,90,CTLT,55.25,0.675505,N/A
6,22,AMZN,183.63,0.672572,N/A
7,85,COF,141.05,0.652031,N/A
8,6,AMD,162.62,0.639322,N/A
9,27,AXP,241.32,0.633794,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio:')

    try:
        float(portfolio_size)
    except ValueError:
        print('That is not a number!')
        print('Please try again')
        portfolio_size = input('Enter the size of your portfolio:')
        

portfolio_input()
print(portfolio_size)

Enter the size of your portfolio: 100000000


100000000


In [9]:
position_size = float(portfolio_size) / len(final_dataframe)
print(position_size)
for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe.loc[i, 'Price'])
final_dataframe

2000000.0


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,44,ANET,320.09,1.362879,6248
1,75,AVGO,1412.13,1.321727,1416
2,40,AMAT,214.03,0.822188,9344
3,34,APH,131.71,0.758475,15184
4,91,CAT,350.72,0.735574,5702
5,90,CTLT,55.25,0.675505,36199
6,22,AMZN,183.63,0.672572,10891
7,85,COF,141.05,0.652031,14179
8,6,AMD,162.62,0.639322,12298
9,27,AXP,241.32,0.633794,8287


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [10]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings[:1]:
    # batch_api_call_url = ''
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        new_data = {
            'Ticker': symbol, 
            'Price': data[symbol]['quote']['latestPrice'], 
            'One-Year Price Return': data[symbol]['stats']['year1ChangePercent'], 
            'Number of Shares to Buy': 'N/A',
            'One-Year Return Percentile': data[symbol]['stats']['year1ChangePercent'],
            'Six-Month Price Return': 'N/A',
            'Six-Month Return Percentile': data[symbol]['stats']['month6ChangePercent'],
            'Three-Month Price Return': 'N/A',
            'Three-Month Return Percentile': data[symbol]['stats']['month3ChangePercent'],
            'One-Month Price Return': 'N/A',
            'One-Month Return Percentile': data[symbol]['stats']['month1ChangePercent'],
            'HQM Score': 'N/A'
        }

        # Get the index for the new row
        new_index = len(hqm_dataframe)
        
        # Set the values for each column at the new index
        for column, value in new_data.items():
            hqm_dataframe.at[new_index, column] = value
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MMM,104.86,N/A,0.071923,0.071923,N/A,0.079748,N/A,0.095908,N/A,0.108872,N/A
1,AOS,85.36,N/A,0.293571,0.293571,N/A,0.165969,N/A,0.07537,N/A,0.02296,N/A
2,ABT,104.87,N/A,-0.028491,-0.028491,N/A,0.07772,N/A,-0.075073,N/A,-0.039489,N/A
3,ABBV,164.35,N/A,0.162688,0.162688,N/A,0.21293,N/A,-0.063822,N/A,0.013113,N/A
4,ACN,308,N/A,0.128852,0.128852,N/A,-0.045021,N/A,-0.166336,N/A,-0.015163,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
95,CE,155.67,N/A,0.53876,0.53876,N/A,0.257296,N/A,0.05221,N/A,0.022626,N/A
96,COR,221.85,N/A,0.282975,0.282975,N/A,0.153059,N/A,-0.046182,N/A,-0.050784,N/A
97,CNC,77.14,N/A,0.161881,0.161881,N/A,0.066366,N/A,-0.002682,N/A,0.073845,N/A
98,CNP,29.94,N/A,0.044046,0.044046,N/A,0.100323,N/A,0.075059,N/A,0.093499,N/A


#### Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [11]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

#Print the entire DataFrame    
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MMM,104.86,N/A,0.071923,0.32,N/A,0.505,N/A,0.505,N/A,0.505,N/A
1,AOS,85.36,N/A,0.293571,0.6,N/A,0.505,N/A,0.505,N/A,0.505,N/A
2,ABT,104.87,N/A,-0.028491,0.2,N/A,0.505,N/A,0.505,N/A,0.505,N/A
3,ABBV,164.35,N/A,0.162688,0.48,N/A,0.505,N/A,0.505,N/A,0.505,N/A
4,ACN,308,N/A,0.128852,0.42,N/A,0.505,N/A,0.505,N/A,0.505,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
95,CE,155.67,N/A,0.53876,0.88,N/A,0.505,N/A,0.505,N/A,0.505,N/A
96,COR,221.85,N/A,0.282975,0.59,N/A,0.505,N/A,0.505,N/A,0.505,N/A
97,CNC,77.14,N/A,0.161881,0.47,N/A,0.505,N/A,0.505,N/A,0.505,N/A
98,CNP,29.94,N/A,0.044046,0.28,N/A,0.505,N/A,0.505,N/A,0.505,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [12]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
]

for row in hqm_dataframe.index:
    momentum_percentiles = [
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] for time_period in time_periods
    ]
    hqm_dataframe.loc[row, 'HQM score'] = mean(momentum_percentiles)

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score,HQM score
0,MMM,104.86,N/A,0.071923,0.32,N/A,0.505,N/A,0.505,N/A,0.505,N/A,0.45875
1,AOS,85.36,N/A,0.293571,0.6,N/A,0.505,N/A,0.505,N/A,0.505,N/A,0.52875
2,ABT,104.87,N/A,-0.028491,0.2,N/A,0.505,N/A,0.505,N/A,0.505,N/A,0.42875
3,ABBV,164.35,N/A,0.162688,0.48,N/A,0.505,N/A,0.505,N/A,0.505,N/A,0.49875
4,ACN,308,N/A,0.128852,0.42,N/A,0.505,N/A,0.505,N/A,0.505,N/A,0.48375
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,CE,155.67,N/A,0.53876,0.88,N/A,0.505,N/A,0.505,N/A,0.505,N/A,0.59875
96,COR,221.85,N/A,0.282975,0.59,N/A,0.505,N/A,0.505,N/A,0.505,N/A,0.52625
97,CNC,77.14,N/A,0.161881,0.47,N/A,0.505,N/A,0.505,N/A,0.505,N/A,0.49625
98,CNP,29.94,N/A,0.044046,0.28,N/A,0.505,N/A,0.505,N/A,0.505,N/A,0.44875


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [13]:
hqm_dataframe.sort_values('HQM score', ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe[:51]
hqm_dataframe.reset_index(inplace = True, drop = True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score,HQM score
0,ANET,320.09,N/A,1.362879,1.0,N/A,0.505,N/A,0.505,N/A,0.505,N/A,0.62875
1,AVGO,1412.13,N/A,1.321727,0.99,N/A,0.505,N/A,0.505,N/A,0.505,N/A,0.62625
2,AMAT,214.03,N/A,0.822188,0.98,N/A,0.505,N/A,0.505,N/A,0.505,N/A,0.62375
3,APH,131.71,N/A,0.758475,0.97,N/A,0.505,N/A,0.505,N/A,0.505,N/A,0.62125
4,CAT,350.72,N/A,0.735574,0.96,N/A,0.505,N/A,0.505,N/A,0.505,N/A,0.61875
5,CTLT,55.25,N/A,0.675505,0.95,N/A,0.505,N/A,0.505,N/A,0.505,N/A,0.61625
6,AMZN,183.63,N/A,0.672572,0.94,N/A,0.505,N/A,0.505,N/A,0.505,N/A,0.61375
7,COF,141.05,N/A,0.652031,0.93,N/A,0.505,N/A,0.505,N/A,0.505,N/A,0.61125
8,AMD,162.62,N/A,0.639322,0.92,N/A,0.505,N/A,0.505,N/A,0.505,N/A,0.60875
9,AXP,241.32,N/A,0.633794,0.91,N/A,0.505,N/A,0.505,N/A,0.505,N/A,0.60625


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [14]:
portfolio_input()

Enter the size of your portfolio: 10000000


In [15]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe.loc[i, 'Price'])
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score,HQM score
0,ANET,320.09,612,1.362879,1.0,N/A,0.505,N/A,0.505,N/A,0.505,N/A,0.62875
1,AVGO,1412.13,138,1.321727,0.99,N/A,0.505,N/A,0.505,N/A,0.505,N/A,0.62625
2,AMAT,214.03,916,0.822188,0.98,N/A,0.505,N/A,0.505,N/A,0.505,N/A,0.62375
3,APH,131.71,1488,0.758475,0.97,N/A,0.505,N/A,0.505,N/A,0.505,N/A,0.62125
4,CAT,350.72,559,0.735574,0.96,N/A,0.505,N/A,0.505,N/A,0.505,N/A,0.61875
5,CTLT,55.25,3548,0.675505,0.95,N/A,0.505,N/A,0.505,N/A,0.505,N/A,0.61625
6,AMZN,183.63,1067,0.672572,0.94,N/A,0.505,N/A,0.505,N/A,0.505,N/A,0.61375
7,COF,141.05,1390,0.652031,0.93,N/A,0.505,N/A,0.505,N/A,0.505,N/A,0.61125
8,AMD,162.62,1205,0.639322,0.92,N/A,0.505,N/A,0.505,N/A,0.505,N/A,0.60875
9,AXP,241.32,812,0.633794,0.91,N/A,0.505,N/A,0.505,N/A,0.505,N/A,0.60625


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [16]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [17]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [18]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy: